This file makes the calls to NASA's API and requests the Chunks with the correct dates

In [1]:
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd
import numpy as np
import shapely
from shapely import Polygon, to_geojson
from dotenv import load_dotenv

api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

load_dotenv()


prods = ['MOD13A1.061', 'MOD15A2H.061', 'DAYMET.004']
layers = [(prods[0], '_500m_16_days_EVI'), (prods[0], '_500m_16_days_NDVI'), (prods[1], "Lai_500m"), (prods[2], 'prcp'), (prods[2], 'tmax'), (prods[2], 'tmin')]

task_type = ['point','area']        # Type of task, area or point
proj = 'geographic'  # Set output projection 
outFormat = ['geotiff', 'netcdf4']  # Set output file format type
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

params = {'limit': 1, 'pretty': True} # Limit API response to the most recent entry, return as pretty json

bounds = np.load("../data-training/ALL.npy", allow_pickle=True)
dates = np.load("../data-training/ALL-dates.npy", allow_pickle=True)

C:\Users\xubil\AppData\Local\Temp\ipykernel_39052\4106748288.py:2: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import getpass, pprint, time, os, cgi, json


In [2]:
bounds.shape

(43433, 4)

In [3]:
curidx = np.load('../Requests/cur_idx.npy')[0]

In [4]:
curidx

42347

In [5]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })

for i in range(1, 5):
    user = os.getenv("EARTHDATA_USERNAME"+str(i))
    password = os.getenv("EARTHDATA_PASSWORD"+str(i))
    token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
    token = token_response['token']                      # Save login token to a variable
    head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

    request_ids = []

    curidxbegan = curidx

    for nextidx in range(curidx, curidx+500):
        bound = bounds[nextidx]
        
        print(nextidx)

        xmin, xmax, ymin, ymax = bound[0], bound[1], bound[2], bound[3]
        # print(xmin, xmax, ymin, ymax)

        poly = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
        features = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(poly)}]

        nps_gc = {"type":"FeatureCollection", "features":features}

        startDate = dates[curidx][5:] + "-" + dates[curidx][0:4]            # Start of the date range for which to extract data: MM-DD-YYYY
        endDate = startDate             # End of the date range for which to extract data: MM-DD-YYYY

        task = {
            'task_type': task_type[1],
            'task_name': "Bounds "+str(curidx),
            'params': {
                'dates': [
                {
                    'startDate': startDate,
                    'endDate': endDate
                }],
                'layers': prodLayer,
                'output': {
                        'format': {
                                'type': outFormat[0]}, 
                                'projection': proj},
                'geo': nps_gc,
            }
        }

        curidx += 1

        task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
        # task_response = {"task_id":"23132930190-4920-194-9"}
        print(task_response)

        if ('task_id' in task_response):
            task_id = task_response['task_id'] 
            request_ids.append(task_id)
        else:
            break
    
    np.save('../Requests/r'+str(i)+"_"+str(curidxbegan)+"-"+str(curidx), request_ids) # Item on right is noninclusive
    
    
np.save('../Requests/cur_idx', [curidx])



# curidx = 5466
# Saves:
# First time: 469 so 470 is next
# Second time: 470 - 966 so 967 is next
# 3: 967 - 1466
# 4: 1467 - 1966
# 5: 1967 - 2466
# 6: 2467 - 2966
# 7: 2967 - 3466
# 8: 3467 - 3751
# 9: 3752 - 3965
# 10: 3966 - 4465
# 11: 4466 - 4965
# 12: 4966 - 5465

42347
{'task_id': 'a1dfb992-9f37-4ef6-94a7-acbf2265c30d', 'status': 'pending'}
42348
{'task_id': '9ac3c6b4-97b5-422c-bad2-2ffe3cb1a17d', 'status': 'pending'}
42349
{'task_id': '00591176-201d-42e1-b0da-fe36a0946fac', 'status': 'pending'}
42350
{'task_id': '5b155b8b-2fea-4975-b950-fa743708bf31', 'status': 'pending'}
42351
{'task_id': 'fda71e46-9f58-4697-bd95-b8a7ff20b0ac', 'status': 'pending'}
42352
{'task_id': '285ed1bf-060a-4ae7-9f16-612a079370c0', 'status': 'pending'}
42353
{'task_id': 'f0a39e51-6e11-4d79-b0af-e26e8b044720', 'status': 'pending'}
42354
{'task_id': '9655fc67-120d-44f8-ad15-fae66b3a12d3', 'status': 'pending'}
42355
{'task_id': 'ad8804d5-d4a1-442d-a3a7-e3f5f02ccc51', 'status': 'pending'}
42356
{'task_id': 'cf7a520e-eb30-4f6c-a27a-0aeeb2446375', 'status': 'pending'}
42357
{'task_id': '2ab6442b-87c6-4d66-b174-ebf2a86685db', 'status': 'pending'}
42358
{'task_id': '110ee83a-dd50-4f3e-b325-371fc870589d', 'status': 'pending'}
42359
{'task_id': 'a2784eb9-7687-4a67-831c-e966d8bc0

IndexError: index 43433 is out of bounds for axis 0 with size 43433

In [8]:
# Final Save Line after Finished Fetching Everything (Code went out of bounds)
np.save('../Requests/r'+str(i)+"_"+str(curidxbegan)+"-"+str(curidx), request_ids)

#### Same code but for missing.npy (some chunks neglected by the above code)

In [4]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })

missing = np.load("../Requests/missing.npy")
curidxmissing = 0

for i in range(1, 2):
    user = os.getenv("EARTHDATA_USERNAME"+str(i))
    password = os.getenv("EARTHDATA_PASSWORD"+str(i))
    token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
    token = token_response['token']                      # Save login token to a variable
    head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

    request_ids = []

    curidxbegan = curidxmissing

    for getnextidx in range(curidxmissing, missing.shape[0]):
        nextidx = missing[getnextidx]
        bound = bounds[nextidx]
        
        print(nextidx)

        xmin, xmax, ymin, ymax = bound[0], bound[1], bound[2], bound[3]
        # print(xmin, xmax, ymin, ymax)

        poly = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
        features = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(poly)}]

        nps_gc = {"type":"FeatureCollection", "features":features}

        startDate = dates[nextidx][5:] + "-" + dates[nextidx][0:4]            # Start of the date range for which to extract data: MM-DD-YYYY
        endDate = startDate             # End of the date range for which to extract data: MM-DD-YYYY

        task = {
            'task_type': task_type[1],
            'task_name': "Bounds "+str(nextidx),
            'params': {
                'dates': [
                {
                    'startDate': startDate,
                    'endDate': endDate
                }],
                'layers': prodLayer,
                'output': {
                        'format': {
                                'type': outFormat[0]}, 
                                'projection': proj},
                'geo': nps_gc,
            }
        }

        # curidx += 1

        task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
        # task_response = {"task_id":"23132930190-4920-194-9"}
        print(task_response)

        if ('task_id' in task_response):
            task_id = task_response['task_id'] 
            request_ids.append(task_id)
        else:
            break
    
    np.save('../Requests/r'+str(i)+"_MISSING_A", request_ids) # Item on right is noninclusive
    
    
np.save('../Requests/cur_idx', [curidx])


25467
{'task_id': '767af532-ea27-4c04-9e5d-a4680fa040b6', 'status': 'pending'}
25942
{'task_id': '6083dffa-4482-4704-9d93-2659bafa65a0', 'status': 'pending'}
